In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

In [6]:

data = pd.read_json('multinli_1.0/multinli_1.0_train.jsonl',lines=True)

In [4]:
data.head()

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],government,neutral,31193n,31193,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...
1,[entailment],telephone,entailment,101457e,101457,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...
2,[entailment],fiction,entailment,134793e,134793,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...
3,[entailment],fiction,entailment,37397e,37397,How do you know? All this is their information...,( ( How ( ( ( do you ) know ) ? ) ) ( ( All th...,(ROOT (S (SBARQ (WHADVP (WRB How)) (SQ (VBP do...,This information belongs to them.,( ( This information ) ( ( belongs ( to them )...,(ROOT (S (NP (DT This) (NN information)) (VP (...
4,[neutral],telephone,neutral,50563n,50563,yeah i tell you what though if you go price so...,( yeah ( i ( ( tell you ) ( what ( ( though ( ...,(ROOT (S (VP (VB yeah) (S (NP (FW i)) (VP (VB ...,The tennis shoes have a range of prices.,( ( The ( tennis shoes ) ) ( ( have ( ( a rang...,(ROOT (S (NP (DT The) (NN tennis) (NNS shoes))...


In [15]:
data['gold_label'].value_counts()

contradiction    130903
neutral          130900
entailment       130899
Name: gold_label, dtype: int64

In [16]:
data["sentence1"]

0         Conceptually cream skimming has two basic dime...
1         you know during the season and i guess at at y...
2         One of our number will carry out your instruct...
3         How do you know? All this is their information...
4         yeah i tell you what though if you go price so...
5         my walkman broke so i'm upset now i just have ...
6         But a few Christian mosaics survive above the ...
7          (Read  for Slate 's take on Jackson's findings.)
8                                        Gays and lesbians.
9         At the end of Rue des Francs-Bourgeois is what...
10        I burst through a set of cabin doors, and fell...
11                             Fun for adults and children.
12        It's not that the questions they asked weren't...
13        Thebes held onto power until the 12th Dynasty,...
14        I don't mean to be glib about your concerns, b...
15                                Issues in Data Synthesis.
16                     well you see that

In [23]:
data["sentence2"]

0         Product and geography are what make cream skim...
1         You lose the things to the following level if ...
2         A member of my team will execute your orders w...
3                         This information belongs to them.
4                  The tennis shoes have a range of prices.
5         I'm upset that my walkman broke and now I have...
6         Most of the Christian mosaics were destroyed b...
7               Slate had an opinion on Jackson's findings.
8                                            Heterosexuals.
9         Place des Vosges is constructed entirely of gr...
10                 I burst through the doors and fell down.
11                                   Fun for only children.
12        All of the questions were interesting accordin...
13        The capital near Memphis lasted only half a ce...
14        I am concerned more about your issues than the...
15                              Problems in data synthesis.
16                 You can see that on t

In [18]:
glove_df = pd.read_csv('glove.6B/glove.6B.300d.txt', sep=" ", quoting=3, header=None, index_col=0)
glove = {key: val.values for key, val in glove_df.T.items()}

In [22]:
replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

def clean_str(x):
    x = str(x)
    
    x = x.lower()
    
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
        
    return x


In [26]:
data['clean_sentence1'] = data['sentence1'].apply(clean_str)

In [27]:
data['clean_sentence2'] = data['sentence2'].apply(clean_str)

In [31]:
! pip install keras 

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl


In [36]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [50]:
X=[]
for i in range(len(data)):
    sent1_words = data["clean_sentence1"][i].split()
    sent2_words = data["clean_sentence2"][i].split()
    X.append(sent1_words+sent2_words)
    

In [57]:
data['sent1_sent2'] = [' '.join(map(str, l)) for l in X]

In [59]:
data['sent1_sent2'][0],y[0]

('conceptually cream skimming has two basic dimensions product and geography product and geography are what make cream skimming work',
 array([0, 0, 1], dtype=uint8))

In [60]:
max_features = 20000 
sequence_length = 100

tokenizer = Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
tokenizer.fit_on_texts(data['sent1_sent2'].values)
X = tokenizer.texts_to_sequences(data['sent1_sent2'].values)

# we then pad the sequences so they're all the same length (sequence_length)
X = pad_sequences(X, sequence_length)


y = pd.get_dummies(data['gold_label']).values

# lets keep a couple of thousand samples back as a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

print("test set size " + str(len(X_test)))

test set size 39271


In [62]:
import os
embeddings_index = {}
f = open(os.path.join('glove.6B', 'glove.6B.300d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400001 word vectors.


In [64]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 70666 unique tokens.


In [65]:
num_words = min(max_features, len(word_index)) + 1
print(num_words)

embedding_dim = 300

# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word, i in word_index.items():
    if i > max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # we found the word - add that words vector to the matrix
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

20001


In [80]:
embedding_matrix[1]

array([-1.53175091e+00, -2.72801742e-01, -3.69787806e-01, -6.20436375e-01,
       -1.41377501e+00, -8.13059165e-01, -5.72278647e-02,  7.47100978e-01,
       -1.33060046e+00,  2.52001646e-01, -7.84653625e-01, -9.50075018e-01,
        1.91752744e+00, -6.25430207e-01,  4.15004878e-01,  6.66200339e-01,
        9.12417930e-01, -9.57297047e-02,  4.39798732e-01,  1.78664677e-01,
       -4.93002171e-01,  1.07069796e+00,  1.83140016e-01, -1.70334308e+00,
       -3.38988314e-01,  1.64378791e+00,  6.11685776e-01, -9.06229523e-01,
        1.53916974e+00, -1.74534094e+00,  4.09513241e-01, -2.33501969e+00,
       -8.40940759e-02,  7.42771337e-01, -1.90792730e-01,  1.46902226e-02,
        1.06540620e+00, -7.16024321e-01,  1.02800659e+00, -1.14940191e+00,
       -8.55198119e-01,  1.32455171e+00,  6.19006167e-01,  1.06995758e-02,
        1.20224386e-01, -6.93915410e-01,  1.86139586e+00, -7.76943913e-01,
       -1.24316148e+00, -5.60003061e-01, -1.10972791e-01, -1.04877114e+00,
       -4.08713749e-01, -

In [83]:
from keras.layers import GRU,Embedding,Dense
from keras.models import Sequential

In [84]:
model = Sequential()
model.add(Embedding(num_words,
                    embedding_dim,
                    weights=[embedding_matrix],
                    input_length=sequence_length,
                    trainable=True))
model.add(GRU(256,activation="tanh",return_sequences=True))
model.add(GRU(128,activation="tanh"))
model.add(Dense(units=3, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 100, 300)          6000300   
_________________________________________________________________
gru_5 (GRU)                  (None, 100, 256)          427776    
_________________________________________________________________
gru_6 (GRU)                  (None, 128)               147840    
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 6,576,303
Trainable params: 6,576,303
Non-trainable params: 0
_________________________________________________________________
None


In [85]:
batch_size = 128
history = model.fit(X_train, y_train, epochs=5, batch_size=batch_size, verbose=1, validation_split=0.1)


Train on 318087 samples, validate on 35344 samples
Epoch 1/5
318087/318087 [==============================] - 2795s 9ms/step - loss: 0.9198 - acc: 0.5539 - val_loss: 0.8624 - val_acc: 0.5978
Epoch 2/5
318087/318087 [==============================] - 3063s 10ms/step - loss: 0.7969 - acc: 0.6393 - val_loss: 0.8277 - val_acc: 0.6226
Epoch 3/5
318087/318087 [==============================] - 3165s 10ms/step - loss: 0.6999 - acc: 0.6956 - val_loss: 0.8640 - val_acc: 0.6135
Epoch 4/5
318087/318087 [==============================] - 9991s 31ms/step - loss: 0.5897 - acc: 0.7524 - val_loss: 0.9306 - val_acc: 0.6080
Epoch 5/5
318087/318087 [==============================] - 2801s 9ms/step - loss: 0.4676 - acc: 0.8107 - val_loss: 1.0723 - val_acc: 0.5914
